In [1]:
import time, onnx, torch, torch, csv
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as utils

from torchvision import transforms
from PIL import Image

## DeepStart network definition

DeepStar consists of two layer of max pooled convolutional neural networks that is then fed into two layers of normal neural networks.

**Get Loss**: Currently we use a custom loss function that is just the difference between the expected and output. Times 10 to keep the learning rate acceptable.<br>
**Get Optim**: Currentl we are using the Adadelta loss function.http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf<br>

In [3]:
class DeepStar(nn.Module):
    def __init__(self):
        super(DeepStar, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=16, stride=2, padding=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=8, stride=2, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=1, padding=0)
        self.pool3 = nn.MaxPool2d(kernel_size=1, stride=1, padding=0)
        
        #self.conv_dropout = nn.Dropout2d()

        self.fc1 = torch.nn.Linear(256 * 12 * 12 + 4, 64)
        self.fc2 = torch.nn.Linear(64 + 4, 2)
        
    def __name__(self):
        return "DeepStar"
    
    def get_loss(self):
        return lambda o ,t: torch.mean(abs(o - t)) * 10
        #return torch.nn.MSELoss()
        
    def get_optim(self, rho, lr, weight_decay):
        return optim.Adadelta(self.parameters(), rho=rho, lr=lr, weight_decay=weight_decay)
        #return optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        
    def forward(self, img, points):
        img = self.forward_conv(img)

        l = img.view(-1, self.conv_out)
        l = torch.cat((l, points), 1)

        l = F.relu(self.fc1(l))
        l = self.dropout1(l)
        
        l = F.relu(self.fc2(l))
        l = self.dropout2(l)

        return self.fc3(l)
    
    def forward_conv(self, img):
        img = F.relu(self.conv1(img))
        img = self.pool1(img)

        img = F.relu(self.conv2(img))
        img = self.pool2(img)
        
        img = F.relu(self.conv3(img))
        img = self.pool3(img)
        
        img = F.relu(self.conv4(img))
        img = self.pool4(img)
        
        img = F.relu(self.conv5(img))
        img = self.pool5(img)
        
        return img

## DataLoader

Because we use a custom dataset we have to have a custom data loader. Currently we only use one map and one csv file for all paths generated on that map. The map is a grayscale heightmap normalzied to between 0 and 1. The data is a list of start and stop points.

In [4]:
class PathDataLoader(utils.Dataset):
    def __init__(self, data_dir):
        self.map = f'{data_dir}map.png'
        self.data_path = f'{data_dir}data.csv'
        self.to_tensor = transforms.ToTensor()
        self.data = pd.read_csv(self.data_path, encoding = "UTF-8")
    
    def __len__(self):
        return len(self.data["Start"])
    
    def __getitem__(self, idx):
        with Image.open(self.map) as img:
            sx, sy = self.to_tuple(self.data["Start"][idx])
            ex, ey = self.to_tuple(self.data["Stop"][idx])
            mx, my = self.to_tuple(self.data["Midpoint"][idx])

            img_tensor = self.to_tensor(img)

            return img_tensor, torch.FloatTensor([sx / 256, sy / 256, ex / 256, ey / 256]), (mx / 256, my / 256)
    
    def to_tuple(self, t):
        return tuple(map(int, t.replace('(','').replace(')', '').split(', '))) 